Here we use a neural network regressor

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import os

In [ ]:
data_path = "../input/tabular-playground-series-aug-2021"

In [ ]:
ss =  os.path.join(data_path, "sample_submission.csv")
ts =  os.path.join(data_path, "test.csv")
tr =  os.path.join(data_path, "train.csv")

In [ ]:
sample_submission = pd.read_csv(ss)
test_file = pd.read_csv(ts)
training_file = pd.read_csv(tr)

In [ ]:
test_file = test_file.drop(labels=["id"], axis=1)

In [ ]:
training_file = training_file.drop(labels=["id"], axis=1)

In [ ]:
test_features = test_file.copy()
test_features = np.array(test_features)

In [ ]:
train_features = training_file.copy()
train_labels = train_features.pop('loss')

In [ ]:
train_features = np.array(train_features)

In [ ]:
num_labels = len(train_labels.unique())

In [ ]:
train_labels =  np.array(train_labels).reshape((train_labels.shape[0], -1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( train_features , train_labels, test_size=0.33, random_state=42)


train_X  = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_X = train_X.shuffle(len(train_X)).batch(32)
validation_X = tf.data.Dataset.from_tensor_slices((X_test, y_test))
validation_X = validation_X.shuffle(len(train_X)).batch(32)

In [ ]:
EPOCHS = 10
checkpoint_filepath = 'regression-model2.{epoch:02d}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq='epoch')

In [ ]:
simple_model = tf.keras.Sequential([
  layers.BatchNormalization(),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.1),
  layers.BatchNormalization(),
  layers.Dense(1024, activation='relu'),
  layers.Dropout(0.1),
  layers.BatchNormalization(),
  layers.Dense(2048, activation='relu'),
  layers.Dropout(0.1),
  layers.BatchNormalization(),
  layers.Dense(2048, activation='relu'),
  layers.Dropout(0.1),
  layers.BatchNormalization(),
  layers.Dense(1024, activation='relu'),
  layers.Dropout(0.1),
  layers.BatchNormalization(),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.1),
  layers.BatchNormalization(),
  layers.Dense(1)
])

In [ ]:
simple_model.compile(optimizer=optimizers.Adam(learning_rate=0.1),
              loss=tf.keras.losses.MSE,
              metrics=['accuracy'])

In [ ]:
simple_model.fit(train_X, validation_data=validation_X,epochs=1, callbacks=[model_checkpoint_callback]) #callbacks=[model_checkpoint_callback]

In [ ]:
simple_model.evaluate(validation_X)

In [ ]:
prediction = simple_model.predict(test_features)

In [ ]:
prediction = prediction.reshape((150000, ))

In [ ]:
sample_submission['loss'] = prediction

In [ ]:
sample_submission.to_csv("submission.csv", index=False)